# Ajuste Exponencial do Circuito RC


## Importações Iniciais

o módulo ajuste contém funções para facilitar a leitura do código atual

In [54]:
import ajuste as aj
import numpy as np
import pandas as pd
import altair as alt

## Parâmetro *Tal*

Estamos buscando encontrar o parâmetro *Tal*, supondo que nossos dados module uma função exponencial de modo:

`a * e^(-b*x) + c`

o parâmetro tal é igual à `1/b`


além de calcular o *Tal* com a regressão adicionaremos uma função adicional que calcula o *Tal* de outra maneira

In [55]:
e = np.exp(1) # Exponencial


def tal_calcul(df:pd.DataFrame) -> int:
    df_reseted = df.reset_index(drop=True) # Resetando index para melhor funcionamento do iloc

    #Buscaremos o valor a/e que representa o momento onde t = tal
    valor_procurado = df_reseted["U_b"].iloc[0]/ e #a/e
    
    #Aqui encontramos o índice do dado mais próximo de a/e
    indice_mais_proximo = (df_reseted["Time"] - valor_procurado).abs().idxmin()

    #Acessando o valor
    valor_mais_proximo = df_reseted["Time"].iloc[indice_mais_proximo]

    return (valor_mais_proximo/1000) #Ajeitando escala de milissegundos para segundos

## Dado e Visualização

Primeiro iremos olhar nossos dados para entender oque está acontecendo

In [56]:
path = "RC-01.csv" 

data = pd.read_csv(path)

#Excluindo dados desnecessários
data = data.drop(columns=["U_y"],axis=1)

aj.graph(data)

alt.Chart(...)

Podemos ver que o referencial do tempo está deslocado, já que começamos a partir do tempo = -0.0025,

mas é fácil arrumar isto basta subtrair este valor à coluna `"Time"` do nossa DataFrame.

Além disso mudaremos a escala de segundos para milissegundos, já que a variação em segundos é muito baixa.

In [57]:
def escale_data(data:pd.DataFrame)->pd.DataFrame:
    data["Time"] -= data["Time"][0]
    data["Time"]*= 1000

escale_data(data)
data.head()

,Time,U_b
0,0.000,0.516000
1,0.005,0.506800
2,0.010,0.485600
3,0.015,0.470267
4,0.020,0.463867


## Regressão

Neste trabalho estamos interessados em demonstrar que tanto a queda de tensão como a ascenção são moduladas por uma função exponencial,

Desta maneira dividiremos a análise em duas partes, primeiro quando a tensão sobe e depois quando a tensão desce


### Tensão subindo:

Primeiramente pegaremos um intervalo de apenas uma onda,

pois assim podemos apenas indentificar a tensão `mínima` e `máxima` e recortar apenas o crescimento

In [58]:
interval = [3.3,4.3]
ini = interval[0]
fin = interval[1]
q = data["Time"].sub(ini).abs().idxmin()
p = data["Time"].sub(fin).abs().idxmin()
semi_data = data[q:p]

aj.graph(semi_data)

alt.Chart(...)

Para isso temos a funçao `slicing_data` do módulo `ajuste`, que filtra a partir de um DataFrame e o intervalo anterior que achamos.

Tendo o dado filtrado podemos facilmente executar a regressão

In [59]:
dado_sub = aj.slicing_data(data,interval,ascending=True) #Dados filtrados

dado_sub_regres, b, cov = aj.regres_exp(dado_sub) #Dados da regressão e o parâmetro b

(aj.data_x_regression(dado_sub,dado_sub_regres)).display() #Gráfico

tal_sb = 1/(b*1000) # Tal é medido em segundos então precisamos ajustar o parâmetro b

alt.LayerChart(...)

### Tensão descendo

Processo igual ao subindo

In [60]:
interval = [1.8,2.6]
ini = interval[0]
fin = interval[1]
q = data["Time"].sub(ini).abs().idxmin()
p = data["Time"].sub(fin).abs().idxmin()
semi_data = data[q:p]

aj.graph(semi_data) #Ignore a escala estamos interessados apenas em ver se pegamos o conjunto de dados certo

alt.Chart(...)

In [61]:
dado_dsc = aj.slicing_data(data, [1.8,2.6])

dado_dsc_regres, c, cov = aj.regres_exp(dado_dsc)

(aj.data_x_regression(dado_dsc,dado_dsc_regres)).display()

tal_dc = 1/(c*1000)
tal_othermethod = tal_calcul(dado_dsc)

alt.LayerChart(...)

Agora queremos salvar nossos resultados!

In [62]:
with open("RC-01.txt", "w") as m:

    m.write(f"tal subindo com ajuste: {tal_sb}\n")
    m.write(f"tal_descendo com ajuste: {tal_dc}\n")
    m.write(f"tal_descendo com mt t=tal: {tal_othermethod}\n")
    


## Repetindo...

Fizemos o circuito RC de duas maneiras, então repetiremos o processo para o outro conjunto de dados

In [63]:
path = "RC-02.csv"
data = pd.read_csv(path)
escale_data(data)

data_dsc2 = aj.slicing_data(data,[0,1.5])
data_dsc2_regres, b, cov = aj.regres_exp(data_dsc2)
(aj.data_x_regression(data_dsc2,data_dsc2_regres)).display()

tal_dsc2 = 1/(b*1000)
tal_carlmethd = tal_calcul(data_dsc2)


alt.LayerChart(...)

In [64]:

data_sb2 = aj.slicing_data(data,[3,10],ascending=True)

data_sb2_regres, b, cov = aj.regres_exp(data_sb2)

(aj.data_x_regression(data_sb2,data_sb2_regres)).display()


tal_sb2 = 1/(b*1000)

alt.LayerChart(...)

In [65]:
with open("RC-02.txt", "w") as m:

    m.write(f"tal subindo com ajuste: {tal_sb2}\n")
    m.write(f"tal_descendo com ajuste: {tal_dsc2}\n")
    m.write(f"tal_descendo com mt t=tal: {tal_carlmethd}\n")
    

print("Arquivo escrito com sucesso!")


Arquivo escrito com sucesso!
